# Agente Consultor de Microsoft Learn vía MCP Remoto

## Descripción

En este ejercicio, desarrollarás un **agente consultor técnico** que utiliza el protocolo MCP (Model Context Protocol) remoto para acceder a documentación técnica de Microsoft Learn y Azure.

### Arquitectura MCP Remoto
```
Pregunta técnica
    ↓
[TechDocsAgent]
    ↓
[MCPStreamableHTTPTool]
    ↓
https://learn.microsoft.com/api/mcp
    ↓
Respuesta basada en documentación Microsoft
```

### Cargar librerías y variables de entorno

In [ ]:
import asyncio
import os
from dotenv import load_dotenv, find_dotenv
from agent_framework import ChatAgent, MCPStreamableHTTPTool
from agent_framework.openai import OpenAIChatClient

load_dotenv(find_dotenv(usecwd=True))

# GitHub Models client configuration
MODEL_NAME = os.getenv("GITHUB_MODEL", "openai/gpt-4o")

### Crear y ejecutar el agente con herramienta MCP remota

Configuramos el `ChatAgent` con instrucciones para usar exclusivamente el servidor MCP de Microsoft Learn. La herramienta `MCPStreamableHTTPTool` se conecta a la API remota.

In [ ]:
async def query_tech_docs(prompt):
    print("Connecting to remote MCP server...")
    
    # GitHub Models client configuration
    client = OpenAIChatClient(
        model_id=MODEL_NAME,
        api_key=os.environ["GITHUB_TOKEN"],
        base_url="https://models.github.ai/inference"
    )
    
    async with (
        MCPStreamableHTTPTool(
            name="mslearn",
            url="https://learn.microsoft.com/api/mcp",
            headers={"Authorization": "Bearer your-token"},  # Example token
        ) as mcp_server,
        ChatAgent(
            chat_client=client,
            name="TechDocsAgent",
            instructions=(
                """
                Eres un agente de IA consultor técnico para documentación de Microsoft y Azure.
                Limita tus respuestas EXCLUSIVAMENTE a la documentación de Microsoft Learn (Microsoft/Azure)
                accesible a través del servidor MCP de Microsoft Learn.
                IMPORTANTE: Mantén tus respuestas concisas, limitándolas a 2-3 líneas como máximo.
                Explica los conceptos con precisión y sugiere pasos prácticos de forma breve.
                Si la pregunta es ambigua, solicita aclaraciones breves. Responde en español por defecto.
                """
            ),
        ) as agent,
    ):
        print(f"Running query: {prompt}")
        result = await agent.run(prompt, tools=mcp_server)
        print("Query result:")
        print(result)


In [ ]:
# Enter your question here
print("Starting MCP remote query...")
user_prompt = "Cómo configurar Azure AI Search para indexar documentación técnica (PDF) y usarlo con Azure OpenAI en un patrón RAG."
await query_tech_docs(user_prompt)

In [ ]:
# Enter your question here
print("Starting MCP remote query...")
user_prompt = "Guía paso a paso para crear un workspace de Azure Machine Learning y desplegar un modelo como endpoint en línea."
await query_tech_docs(user_prompt)